In [ ]:
from django.shortcuts import render
from django.http import HttpResponse, JsonResponse
from django.views import View
from django.template.loader import render_to_string
from django.core.mail import send_mail
from django.conf import settings
from django.utils.crypto import get_random_string
from django.utils import timezone
from django.contrib.auth import login
from django.shortcuts import redirect
from datetime import timedelta
import uuid
import logging

logger = logging.getLogger(__name__)

class MagicLinkView(View):
    """
    View for handling magic link authentication
    """
    def get(self, request):
        """
        Display the form to request a magic link
        """
        return render(request, 'magic_link_form.html')
    
    def post(self, request):
        """
        Process the request for a magic link
        """
        email = request.POST.get('email')
        
        if not email:
            return JsonResponse({'error': 'Email is required'}, status=400)
        
        # Get or create user based on email
        try:
            from django.contrib.auth import get_user_model
            User = get_user_model()
            user, created = User.objects.get_or_create(
                email=email,
                defaults={'username': email.split('@')[0]}
            )
        except Exception as e:
            logger.error(f"Error finding/creating user: {str(e)}")
            return JsonResponse({'error': 'Could not process your request'}, status=500)
        
        # Generate token and store in session or database
        token = str(uuid.uuid4())
        
        # Store the token with expiration time (5 minutes)
        request.session[f'magic_link_{token}'] = {
            'user_id': user.id,
            'expires_at': (timezone.now() + timedelta(minutes=5)).timestamp()
        }
        
        # Generate magic link URL
        login_url = f"{request.scheme}://{request.get_host()}/auth/verify-magic-link/{token}/"
        
        # Email context
        context = {
            'user': user,
            'login_url': login_url,
            'app_name': getattr(settings, 'APP_NAME', 'Our Application')
        }
        
        # Render email template
        email_html = render_to_string('magic_link_email.html', context)
        
        # Send email
        try:
            send_mail(
                f'Your login link for {context["app_name"]}',
                f'Hello {user.username}, here is your login link: {login_url}',
                settings.DEFAULT_FROM_EMAIL,
                [email],
                html_message=email_html,
                fail_silently=False
            )
            return JsonResponse({'success': 'Magic link sent to your email'})
        except Exception as e:
            logger.error(f"Error sending email: {str(e)}")
            return JsonResponse({'error': 'Could not send email'}, status=500)


class VerifyMagicLinkView(View):
    """
    View for verifying a magic link and logging in the user
    """
    def get(self, request, token):
        # Check if token exists and is valid
        session_key = f'magic_link_{token}'
        token_data = request.session.get(session_key)
        
        if not token_data:
            return render(request, 'magic_link_error.html', {
                'error': 'Invalid or expired link'
            })
        
        # Check if token has expired
        expires_at = token_data.get('expires_at')
        if expires_at and expires_at < timezone.now().timestamp():
            # Clean up expired token
            del request.session[session_key]
            return render(request, 'magic_link_error.html', {
                'error': 'This login link has expired'
            })
        
        # Get the user
        try:
            from django.contrib.auth import get_user_model
            User = get_user_model()
            user = User.objects.get(id=token_data.get('user_id'))
            
            # Log the user in
            login(request, user)
            
            # Clean up used token
            del request.session[session_key]
            
            # Redirect to home or dashboard
            redirect_url = request.GET.get('next', settings.LOGIN_REDIRECT_URL)
            return redirect(redirect_url)
            
        except User.DoesNotExist:
            return render(request, 'magic_link_error.html', {
                'error': 'User not found'
            })
        except Exception as e:
            logger.error(f"Error during magic link verification: {str(e)}")
            return render(request, 'magic_link_error.html', {
                'error': 'An error occurred during login'
            })